## [Running Flask and FastAPI on Google Colab](https://medium.datadriveninvestor.com/flask-on-colab-825d2099d9d8)

In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn

     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 747kB 19.0MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 10.1MB 23.3MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19246 sha256=45c54fb442ad56748641ba96f2de35c12ee50b17e98a32dbd65a3b10424ec485
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok
  Found existing installation: click 8.0.0
    Uninstalling click-8.0.0:
      Successfully uninstalled click-8.0.0


In [3]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn

app = FastAPI()

@app.get('/index')
async def home():
  return "Hello World"

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
print('Public docs URL:', ngrok_tunnel.public_url + "/docs")
nest_asyncio.apply()
uvicorn.run(app, port=8000)


INFO:     Started server process [58]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://838876fd6123.ngrok.io
Public docs URL: http://838876fd6123.ngrok.io/docs
INFO:     124.32.186.1:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     124.32.186.1:0 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [58]


## [Deploying Transformer Models](https://chatbotslife.com/deploying-transformer-models-1350876016f)


In [4]:
!pip install -qq transformers

     |████████████████████████████████| 2.3MB 10.8MB/s 
     |████████████████████████████████| 3.3MB 37.3MB/s 
     |████████████████████████████████| 901kB 34.8MB/s 


In [5]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd '/content/drive/My Drive/work/'

/content/drive/My Drive/work


In [7]:
#!pip install datasets
#!pip install rouge_score
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 10.0MB/s 


In [8]:
import torch
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSeq2SeqLM
)

class NLP:
    def __init__(self):
        self.gen_model = AutoModelForSeq2SeqLM.from_pretrained('output/')
        self.gen_tokenizer = AutoTokenizer.from_pretrained('sonoisa/t5-base-japanese')
        self.gen_model.eval()
         
    def summarize(self, prompt="The epistemelogical limit"):
        inputs = self.gen_tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
        with torch.no_grad():
            summary_ids = self.gen_model.generate(inputs) #, max_length=512, min_length=5, length_penalty=5., num_beams=2)
            summary = self.gen_tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
            return summary

In [9]:
# from nlp import NLP
nlp = NLP()

text = r"""
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女ブルマと出会う。そこで、7つ集めると神龍が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャなどを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。
"""

print(nlp.summarize(text))



ドラゴンボールを探す旅に出た孫悟空は、7つ集めると神龍が現れるというドラゴンボールを探す旅に出る。


In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
#from app.nlp import NLP

class Message(BaseModel):
    input: str
    output: str = None

app = FastAPI()
nlp = NLP()

origins = [
    "http://localhost",
    "http://localhost:3000",
    "http://127.0.0.1:3000"
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["POST"],
    allow_headers=["*"],
)

@app.post("/summarize/")
async def  generate(message: Message):
    message.output  = nlp.summarize(prompt=message.input)
    return {"output" : message.output}

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
print('Public docs URL:', ngrok_tunnel.public_url + "/docs")
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://4b100cdd3e6e.ngrok.io
Public docs URL: http://4b100cdd3e6e.ngrok.io/docs


INFO:     Started server process [57]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     124.32.186.1:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     124.32.186.1:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     124.32.186.1:0 - "POST /summarize/ HTTP/1.1" 200 OK
INFO:     124.32.186.1:0 - "POST /summarize/ HTTP/1.1" 200 OK
INFO:     124.32.186.1:0 - "POST /summarize/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [57]


```
% curl -X 'POST' \
  'http://4b100cdd3e6e.ngrok.io/summarize/' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "input": "地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女ブルマと出会う。そこで、7つ集めると神龍が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャなどを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。"
}'

{"output":"願いを一つだけ叶えてくれる「四星球」を探す旅に出る。"}
```